In [1]:
import numpy as np

from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import EarlyStopping

from raims import FixedSizedDataModule, MultilayerPerceptronModel

In [3]:
seed = np.random.SeedSequence(42)

data = FixedSizedDataModule('../data/splits/mona-random', max_mz=1001, seed=seed)

2022-03-09 10:38:46,832:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,842:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,853:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,860:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,866:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,874:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,886:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,891:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,898:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,907:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2022-03-09 10:38:46,916:WARNING:matchms:add_precursor_mz:No precursor_mz found i

In [4]:
mlp_1 = MultilayerPerceptronModel(max_mz=1001, hidden_layers=[1000], learning_rate=1e-3)
mlp_2 = MultilayerPerceptronModel(max_mz=1001, hidden_layers=[1000, 1000], learning_rate=1e-3)
mlp_3 = MultilayerPerceptronModel(max_mz=1001, hidden_layers=[1000, 1000, 1000], learning_rate=5e-4)

mlp_1s = MultilayerPerceptronModel(max_mz=1001, hidden_layers=[500], learning_rate=1e-3)
mlp_2s = MultilayerPerceptronModel(max_mz=1001, hidden_layers=[500, 500], learning_rate=1e-3)
mlp_3s = MultilayerPerceptronModel(max_mz=1001, hidden_layers=[500, 500, 500], learning_rate=1e-3)

In [6]:
def get_num_trainable_params(model: LightningModule):
    return sum(par.numel() for par in model.parameters() if par.requires_grad)

print(f'mlp_1 trainable params: {get_num_trainable_params(mlp_1)}')
print(f'mlp_2 trainable params: {get_num_trainable_params(mlp_2)}')
print(f'mlp_3 trainable params: {get_num_trainable_params(mlp_3)}')

print(f'mlp_1s trainable params: {get_num_trainable_params(mlp_1s)}')
print(f'mlp_2s trainable params: {get_num_trainable_params(mlp_2s)}')
print(f'mlp_3s trainable params: {get_num_trainable_params(mlp_3s)}')

mlp_1 trainable params: 2004001
mlp_2 trainable params: 3005001
mlp_3 trainable params: 4006001
mlp_1s trainable params: 1002501
mlp_2s trainable params: 1253001
mlp_3s trainable params: 1503501


In [7]:
trainer =  Trainer(callbacks=[EarlyStopping(monitor='val_loss', patience=10)], max_epochs=500)

trainer.fit(model=mlp_1, datamodule=data)
trainer.fit(model=mlp_2, datamodule=data)
trainer.fit(model=mlp_3, datamodule=data)

trainer.fit(model=mlp_1s, datamodule=data)
trainer.fit(model=mlp_2s, datamodule=data)
trainer.fit(model=mlp_3s, datamodule=data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: /workspace/notebooks/lightning_logs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 2.0 M 
-------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.016     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:659: UserWarning: Your `val_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(


AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/raims/data.py", line 152, in __getitem__
    binned_spectrum = _spectrum_binning(spectrum_document, self.size)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataset.py", line 83, in __getattr__
    raise AttributeError
AttributeError
